In [ ]:
import os
import subprocess
import sys
import fileinput
import time
from datetime import datetime
import requests
import ipywidgets as widgets
from IPython.display import clear_output
from IPython.utils import capture
from subprocess import getoutput
from PIL import Image
from IPython.display import display, HTML, Image
from io import BytesIO
import threading
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("Link sever sử dụng Online:", l[l.find("http"):], end='')

def sever_flare(port):
  threading.Thread(target=iframe_thread, daemon=True, args=(port,)).start()

def inf(msg, style, wdth):
    inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(inf)
def aria_down(link,path,name):
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d  {path} -o {name}

def c_folder(path):
  path = f'{User_folder}/{path}'
  os.makedirs(path, exist_ok=True)

def default_file_check(folder_source,folder_check,over=False):
  for item in os.listdir(folder_source):
    file_path = os.path.join(folder_source, item)
    if os.path.isfile(file_path):
      cp_path = file_path.replace(SDVNFolder, folder_check)
      !cp {'-f' if over else ''} {file_path} {cp_path}   
    elif os.path.isdir(file_path):
      default_file_check(file_path,folder_check,over)

def run_list_txt(txt):
  with open(f"{txt}", "r") as file:
    lists = file.readlines()
  for list in lists :
    subprocess.run(ex,shell=True)

def link_folder(source,target):
  !ln -s {source} {target}

def split_string_at(link):
    delimiter = "@="
    if delimiter in link:
        parts = link.split(delimiter)
        return parts[0], f'{parts[1]}.safetensors'
    else:
        return link, None

def download(link):
  link = link.replace('&', '\&')
  valuelink, valuename = split_string_at(link)
  if '#' not in link:
    if valuename != None :
      !wget -c {valuelink} -O {checkpointpatch}/{valuename}
    else :
      !wget {"" if "?token=" in valuelink else "-c"} {valuelink} -P {checkpointpatch} --content-disposition

def download_lib(model):
  downloadcode = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
  if 'https:' in model :
    download(model)
  elif '.ckpt' not in model and '.gguf' not in model:
    model += '.safetensors'
    !{downloadcode} {modellist[model]} -d {checkpointpatch} -o {model}

def download_txt(lines):
  for line in lines:
    download(line)

def run_ver(Version,CommandLine):
  %cd {Version_folder}
  if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2":
    sever_flare(7860)
    !python launch.py --xformers --theme dark --enable-insecure-extension-access --disable-safe-unpickle --share --no-half-vae {CommandLine}\
      --ui-settings-file {User_folder}/Setting/config.json \
      --ui-config-file {User_folder}/Setting/ui-config.json \
      --styles-file {User_folder}/Setting/styles.csv --lora-dir {User_folder}/Lora\
      {f"--ckpt-dir {User_folder}/Model --embeddings-dir {User_folder}/Embeddings" if DriveLib else ""}
  if Version == "ComfyUI" :
    !python main.py --dont-print-server --preview-method auto --port 8888 --input-directory {User_folder}/ComfyUIinput \
    {'--front-end-version Comfy-Org/ComfyUI_legacy_frontend@latest' if ComfyLegacy else ''} {CommandLine}
  if Version == "Kohya" :
    !python {Version_folder}/kohya_gui.py --share --headless
  if Version == "Fooocus" :
    !python entry_with_update.py --share --theme dark --disable-preset-download {CommandLine}
  if Version == "AutoRetouch":
    !python {Version_folder}/auto_retoucher.py